In [ ]:
import numpy as np
import pandas as pd
import re
pd.set_option('max_colwidth', 900)
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

from warnings import filterwarnings
filterwarnings("ignore")

df = pd.read_excel("raw_data.xlsx",sheet_name='raw_data')
df['product_name'] = df['product_name'].apply(lambda x : x.replace('"',""))

### removing the Duplicates
df_duplicate_dropped = df.drop_duplicates()
df_duplicate_dropped = df_duplicate_dropped.reset_index()
df_duplicate_dropped = df_duplicate_dropped.drop(columns={"index"})

### Seperating the data for cleaning

df_duplicate_dropped['no_ofprice']      = df_duplicate_dropped['product_price'].apply(lambda x : len(str(x).split(",")))
df_duplicate_dropped['no_of_prods']     = df_duplicate_dropped['product_quantity'].apply(lambda x : len(str(x).split("|")))
df_duplicate_dropped["no_of_prod_name"] = df_duplicate_dropped["product_name"].apply(lambda x: len(str(x).split(",")))


df_no_need_clean = df_duplicate_dropped[(df_duplicate_dropped["no_of_prods"]==df_duplicate_dropped["no_ofprice"])&\
                                       (df_duplicate_dropped["no_of_prods"]==df_duplicate_dropped["no_of_prod_name"])&\
                                       (df_duplicate_dropped["no_ofprice"] ==df_duplicate_dropped["no_of_prod_name"])]

df_need_clean   = df_duplicate_dropped[~((df_duplicate_dropped["no_of_prods"]==df_duplicate_dropped["no_ofprice"])&\
                                       (df_duplicate_dropped["no_of_prods"]==df_duplicate_dropped["no_of_prod_name"])&\
                                       (df_duplicate_dropped["no_ofprice"] ==df_duplicate_dropped["no_of_prod_name"]))]

df_no_need_clean = df_no_need_clean.reset_index()
df_no_need_clean = df_no_need_clean.drop(columns={"index"})

df_need_clean = df_need_clean.reset_index()
df_need_clean = df_need_clean.drop(columns={"index"})

In [182]:
### Cleaning the price
df_equal = df_need_clean[df_need_clean['no_of_prods']==df_need_clean['no_of_prod_name']]

cols = ["no_ofprice","no_of_prods","no_of_prod_name"]
df_main_1 = df_equal[((df_equal['no_ofprice']==2)&(df_equal['no_of_prods']==1))].drop(cols, axis =1)
df_main_1["product_price"] = df_main_1["product_price"].apply(lambda x : int(str(x).replace(",","")))

df_equal = df_equal[~((df_equal['no_ofprice']==2)&(df_equal['no_of_prods']==1))].drop(cols, axis =1)
df_equal = df_equal.reset_index()
df_equal = df_equal.drop(columns={"index"})

#########################################
df_not_equal = df_need_clean[~(df_need_clean['no_of_prods']==df_need_clean['no_of_prod_name'])]

cols = ["no_ofprice","no_of_prods","no_of_prod_name"]
df_main_2 = df_not_equal[(df_not_equal['no_ofprice']==1)&(df_not_equal['no_ofprice']==df_not_equal['no_of_prods'])].drop(cols, axis =1)

df_not_equal = df_not_equal[~((df_not_equal['no_ofprice']==1)&(df_not_equal['no_ofprice']==df_not_equal['no_of_prods']))]
df_not_equal = df_not_equal.reset_index()
df_not_equal = df_not_equal.drop("index", axis = 1)

In [209]:
# df_main_3 = pd.DataFrame()
# val     = []
# for i in range(len(df_no_need_clean)):
#     datereceived = df_no_need_clean['datereceived'][i]
#     user_id      = df_no_need_clean['user_id'][i]
#     customer_id  = df_no_need_clean['customer_id'][i]
#     total_revenue = df_no_need_clean["total_revenue"][i]
#     product_price = df_no_need_clean['product_price'][i]
#     product_quantity = df_no_need_clean['product_quantity'][i]
#     product_name = df_no_need_clean['product_name'][i]
    
#     product_price_cal    = str(product_price).split(",")
#     product_quantity_cal = str(product_quantity).split("|")
#     product_name_cal     = str(product_name).split(",")

#     n1 = len(product_price_cal)
#     n2 = len(product_quantity_cal)
#     n3 = len(product_name_cal)
    
#     if (n3 > n1) & (n1==1): 
#         product_price    = str(product_price).split(",")
#         product_quantity = str(product_quantity).split("|")
#         print(str(product_name))
#         product_name     = str(product_name)
#     else:
#         product_price    = str(product_price).split(",")
#         product_quantity = str(product_quantity).split("|")
#         product_name     = str(product_name).split(",")
        
#     data = {"datereceived" : datereceived,
#               "user_id" : user_id,
#                "total_revenue" : total_revenue, 
#               "customer_id" : customer_id,
#               "product_price" : product_price,
#               "product_quantity" : product_quantity,
#               "product_name" : product_name}

#     df_temp = pd.DataFrame(data)
#     df_main_3 = pd.concat([df_main_3, df_temp], axis = 0) 

# df_main_3.to_csv("cleaned_data.csv", header = True, index = False)
df_main_3 = pd.read_csv("cleaned_data.csv")
df_main_4 = pd.concat([pd.concat([df_main_1,df_main_2],axis = 0),df_main_3],axis = 0)

In [210]:
### product information
df_product_info     = df_main_4[["product_price","product_name"]].drop_duplicates()
df_product_distinct = pd.DataFrame(pd.Series(df_product_info["product_name"].value_counts()))
df_product_distinct = df_product_distinct.reset_index().rename(columns = {"index":"product_name",
                                                                        "product_name":"count"})
df_product_distinct = df_product_distinct[df_product_distinct['count']==1]
df_product = df_product_distinct.merge(df_product_info, on=['product_name'], how= 'left')
df_product = df_product.drop(["count"],axis =1)

In [211]:
### df_equal

df_equal_cleaned = pd.DataFrame()

for i in range(len(df_equal)):
    datereceived     = df_equal['datereceived'][i]
    user_id          = df_equal['user_id'][i]
    customer_id      = df_equal['customer_id'][i]
    total_revenue    = df_equal['total_revenue'][i]
    product_quantity = df_equal['product_quantity'][i]
    product_name     = df_equal['product_name'][i]
    
    product_quantity = str(product_quantity).split("|")
    product_name    = str(product_name).split(",")
    
    data = {"datereceived" : datereceived,
              "user_id" : user_id,
               "total_revenue" : total_revenue, 
              "customer_id" : customer_id,
              "product_quantity" : product_quantity,
              "product_name" : product_name}

    df_temp = pd.DataFrame(data)
    df_equal_cleaned = pd.concat([df_equal_cleaned, df_temp], axis = 0)
    
df_equal_cleaned['original_product_name'] = df_equal_cleaned['product_name']
df_equal_cleaned['product_name'] = df_equal_cleaned['product_name'].apply(lambda x : x.strip().lower())
df_product['product_name'] = df_product['product_name'].apply(lambda x : x.strip().lower())

df_equal = df_equal_cleaned.merge(df_product, how = 'left', on = 'product_name')
df_equal = df_equal.drop("product_name", axis = 1)
df_equal = df_equal.rename(columns = {"original_product_name":"product_name"})

df_equal_left = df_equal[df_equal['product_price'].isna()]
df_main_5     = df_equal[~(df_equal['product_price'].isna())]

df_main_6 = pd.concat([df_main_4,df_main_5],axis = 0)

KeyboardInterrupt: 

In [ ]:
### df_not_equal

df_not_equal["diff"]      = df_not_equal['no_of_prod_name']-df_not_equal['no_ofprice']
df_not_equal['value_map'] = df_not_equal['product_name'].apply(lambda x : [len(i) for i in str(x).split(",")])

df_temp_not_equal = pd.DataFrame()

for i in range(len(df_not_equal)):
    datereceived  = df_not_equal['datereceived'][i]
    user_id       = df_not_equal['user_id'][i]
    customer_id   = df_not_equal['customer_id'][i]
    total_revenue = df_not_equal['total_revenue'][i]
    product_price = df_not_equal['product_price'][i]
    product_name  = df_not_equal['product_name'][i]
    product_quantity = df_not_equal['product_quantity'][i]
    
    product_price = str(product_price).split(",")
    product_quantity = str(product_quantity).split("|")
    
    diff = df_not_equal['diff'][i]
    value_map = df_not_equal['value_map'][i]
    value_map.sort()
    remove = value_map[:diff]
    
    product_names = [i.strip() for i in str(product_name).split(",") if len(i) not in remove]
    
    data = {"datereceived" : datereceived,
            "user_id" : user_id,
            "total_revenue" : total_revenue, 
            "customer_id" : customer_id,
            "product_price" : product_price,
            "product_quantity" : product_quantity,
            "product_name" : product_names}

    df_temp = pd.DataFrame(data)
    df_temp_not_equal = pd.concat([df_temp_not_equal, df_temp], axis = 0)
    
df_temp_not_equal["product_price"] = df_temp_not_equal["product_price"].apply(lambda x : float(x))
df_main_7 = df_temp_not_equal[df_temp_not_equal["product_price"]>0]
df_main_8 = pd.concat([df_main_6,df_main_7],axis = 0)

In [ ]:
### df_equal_left

df_equal_left['product_quantity']  = df_equal_left['product_quantity'].apply(lambda x : int(x))
df_equal_left = df_equal_left.drop("product_price",axis=1)

df_main_8['datereceived']       = pd.to_datetime(df_main_8['datereceived']).dt.date
df_equal_left['datereceived']   = pd.to_datetime(df_equal_left['datereceived']).dt.date

cols = ["datereceived","product_price","product_name"]
full_products  = df_main_8[cols].drop_duplicates()

daw = pd.DataFrame()

In [207]:
df_main_8['year'] = df_main_8['datereceived'].year
df_main_8['month'] = df_main_8['month'].month

df_equal_left['year']  = df_equal_left['datereceived'].year
df_equal_left['month'] = df_equal_left['month'].month

AttributeError: 'Series' object has no attribute 'year'

In [199]:
df_main_8[(df_main_8['user_id']==17687)&(df_main_8['customer_id']==1085159)]

,datereceived,user_id,customer_id,total_revenue,product_price,product_quantity,product_name
20,2021-04-01,17687,1085159,8211.0,899.0,1,Euclidean Black Etagere


In [205]:
a = df_duplicate_dropped[(df_duplicate_dropped['user_id']==17687)&(df_duplicate_dropped['customer_id']==1085159)]
df_main_8[(df_main_8['product_name'].isin(str(a.product_name).split(",")))][['product_name','product_price']].drop_duplicates()

,product_name,product_price
421,Sunset Teak Outdoor Loveseat,1499
2875,Sunset Teak Outdoor Loveseat,1599


In [201]:
df_equal_left[(df_equal_left['user_id']==17687)&(df_equal_left['customer_id']==1085159)]

,datereceived,user_id,total_revenue,customer_id,product_quantity,product_name
18,2021-04-01,17687,8211.0,1085159,6,Sunset Teak Lounge Chair
19,2021-04-01,17687,8211.0,1085159,2,Sunset Teak Outdoor Loveseat


In [202]:
df_duplicate_dropped[(df_duplicate_dropped['user_id']==17687)&(df_duplicate_dropped['customer_id']==1085159)]

,datereceived,user_id,customer_id,total_revenue,product_price,product_quantity,product_name,no_ofprice,no_of_prods,no_of_prod_name
249,4/1/2021 3:19,17687,1085159,8211.0,"7,191,499,899",6|2|1,"Sunset Teak Lounge Chair,Sunset Teak Outdoor Loveseat,Euclidean Black Etagere",4,3,3
